In [1]:
# import os
# import pandas as pd
# from shutil import copy2

# def convert_to_yolo(row, width, height):
#     x1 = row['Roi.X1']
#     y1 = row['Roi.Y1']
#     x2 = row['Roi.X2']
#     y2 = row['Roi.Y2']

#     box_width = x2 - x1
#     box_height = y2 - y1
#     center_x = x1 + box_width / 2
#     center_y = y1 + box_height / 2

#     center_x /= width
#     center_y /= height
#     box_width /= width
#     box_height /= height

#     return center_x, center_y, box_width, box_height

# def process_csv(csv_path, images_root_dir, output_images_dir, output_labels_dir):
#     data_csv = pd.read_csv(csv_path)
#     for index, row in data_csv.iterrows():
#         image_path = row['Path']
#         image_name = os.path.basename(image_path)
#         txt_name = image_name.replace('.png', '.txt')

#         width, height = row['Width'], row['Height']
#         center_x, center_y, box_width, box_height = convert_to_yolo(row, width, height)
#         cls = row['ClassId']

#         src = os.path.join(images_root_dir, image_path)
#         dst = os.path.join(output_images_dir, image_name)

#         os.makedirs(output_images_dir, exist_ok=True)

#         if os.path.exists(src):
#             copy2(src, dst)
#         else:
#             continue

#         label_path = os.path.join(output_labels_dir, txt_name)
#         os.makedirs(output_labels_dir, exist_ok=True)
#         with open(label_path, 'w') as f:
#             f.write(f'{cls} {center_x} {center_y} {box_width} {box_height}\n')

# train_csv_path = '../input/gtsrb-german-traffic-sign/Train.csv'
# test_csv_path = '../input/gtsrb-german-traffic-sign/Test.csv'
# images_root_dir = '../input/gtsrb-german-traffic-sign'  
# output_train_images_dir = 'datasets/train/images'
# output_train_labels_dir = 'datasets/train/labels'
# output_test_images_dir = 'datasets/val/images'
# output_test_labels_dir = 'datasets/val/labels'

# process_csv(train_csv_path, images_root_dir, output_train_images_dir, output_train_labels_dir)
# process_csv(test_csv_path, images_root_dir, output_test_images_dir, output_test_labels_dir)


In [2]:
# !pip install ultralytics==8.0.196

# from IPython import display
# display.clear_output()

# import ultralytics
# ultralytics.checks()

In [3]:
# !nvidia-smi

In [4]:
# !yolo detect mode=train data=/kaggle/input/data-yaml/data.yaml model=yolov8s.yaml epochs=20 imgsz=512 batch=64

In [5]:
# !yolo detect train data=/kaggle/input/data-yaml/data.yaml model=yolov8n.pt epochs=20 imgsz=800 batch=64


In [6]:
# import os
# from ultralytics import YOLO

# data_path = '/kaggle/input/data-yaml/data.yaml'
# model_path = 'yolov8s.pt'

# model = YOLO(model_path)

# model.train(
#     data=data_path,
#     epochs=40,
#     imgsz=640,
#     batch=64,
#     fliplr=0.0,    # Hindari flipping horizontal (mirroring)
#     flipud=0.0,    # Hindari flipping vertikal (mirroring)
#     degrees=0.0,   # Hindari rotasi
# )


# Bagian II

## Latih Model dengan Dataset yg Beragam (Dataset dengan Jarak Jauh dan lebih dari 1 Kelas pada 1 Gambar)

In [7]:
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.32 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 5689.3/8062.4 GB disk)


In [8]:
import os
import shutil
import cv2
from ultralytics import YOLO

# Muat model YOLOv8 yang sudah dilatih
model = YOLO('/kaggle/input/trafficsign/other/yolo/1/best11.pt')  # Ganti dengan path ke model YOLOv8 Anda

# Fungsi untuk memperbesar bounding box
def enlarge_bbox(bbox, scale=1.2):
    center_x, center_y, width, height = bbox
    new_width = width * scale
    new_height = height * scale
    return [center_x, center_y, new_width, new_height]

# Fungsi untuk memprediksi dan memperbarui label
def predict_and_update_label(image_path, label_path, model, output_image_path, output_label_path):
    image = cv2.imread(image_path)
    height, width = image.shape[:2]

    # Baca label yang ada
    with open(label_path, 'r') as file:
        lines = file.readlines()

    new_labels = []

    for line in lines:
        parts = line.strip().split()
        class_id = int(parts[0])
        center_x = float(parts[1]) * width
        center_y = float(parts[2]) * height
        bbox_width = float(parts[3]) * width
        bbox_height = float(parts[4]) * height

        # Perbesar bounding box
        bbox = [center_x, center_y, bbox_width, bbox_height]
        enlarged_bbox = enlarge_bbox(bbox)

        # Crop image sesuai bounding box yang diperbesar
        x_min = max(0, int(enlarged_bbox[0] - enlarged_bbox[2] / 2))
        y_min = max(0, int(enlarged_bbox[1] - enlarged_bbox[3] / 2))
        x_max = min(width, int(enlarged_bbox[0] + enlarged_bbox[2] / 2))
        y_max = min(height, int(enlarged_bbox[1] + enlarged_bbox[3] / 2))

        cropped_image = image[y_min:y_max, x_min:x_max]

        # Prediksi menggunakan YOLOv8 pada gambar yang sudah di-crop
        results = model(cropped_image)

        for result in results:
            for box in result.boxes:
                cls = box.cpu().cls.numpy()[0]
                conf = box.cpu().conf.numpy()[0]
                x_center, y_center, w, h = box.cpu().xywh.numpy()[0]

                # Transformasi kembali koordinat ke skala asli gambar
                x_center = (x_center + x_min) / width
                y_center = (y_center + y_min) / height
                w /= width
                h /= height

                if conf > 0.5:  # Confidence threshold
                    new_labels.append(f'{int(cls)} {x_center} {y_center} {w} {h}')

    # Simpan hasil prediksi baru
    with open(output_label_path, 'w') as out_file:
        for label in new_labels:
            out_file.write(label + '\n')

    # Simpan gambar asli ke direktori output
    cv2.imwrite(output_image_path, image)

# Path ke direktori dengan gambar dan label
data_dir = '/kaggle/input/traffic-signs-dataset-in-yolo-format/ts/ts'
output_dir = '/kaggle/working/dataset2'  # Ganti dengan path ke direktori output

# Buat direktori output jika belum ada
test_dir = os.path.join(output_dir, 'test')
train_dir = os.path.join(output_dir, 'train')
os.makedirs(test_dir, exist_ok=True)
os.makedirs(train_dir, exist_ok=True)

# Baca daftar file test dari test.txt
with open('/kaggle/input/traffic-signs-dataset-in-yolo-format/test.txt', 'r') as file:
    test_files = {os.path.basename(line.strip()) for line in file.readlines()}

# Proses setiap file dalam direktori
for filename in os.listdir(data_dir):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(data_dir, filename)
        label_path = os.path.join(data_dir, os.path.splitext(filename)[0] + '.txt')
        
        if filename in test_files:
            output_image_path = os.path.join(test_dir, filename)
            output_label_path = os.path.join(test_dir, os.path.splitext(filename)[0] + '.txt')
        else:
            output_image_path = os.path.join(train_dir, filename)
            output_label_path = os.path.join(train_dir, os.path.splitext(filename)[0] + '.txt')
        
        predict_and_update_label(image_path, label_path, model, output_image_path, output_label_path)



0: 640x640 1 class_18, 17.2ms
Speed: 11.5ms preprocess, 17.2ms inference, 745.8ms postprocess per image at shape (1, 3, 640, 640)

0: 544x640 1 class_11, 81.3ms
Speed: 1.6ms preprocess, 81.3ms inference, 1.5ms postprocess per image at shape (1, 3, 544, 640)

0: 608x640 1 class_13, 78.7ms
Speed: 2.9ms preprocess, 78.7ms inference, 1.2ms postprocess per image at shape (1, 3, 608, 640)

0: 608x640 1 class_13, 16.3ms
Speed: 2.5ms preprocess, 16.3ms inference, 1.1ms postprocess per image at shape (1, 3, 608, 640)

0: 608x640 1 class_13, 16.3ms
Speed: 2.5ms preprocess, 16.3ms inference, 1.1ms postprocess per image at shape (1, 3, 608, 640)

0: 640x640 1 class_2, 17.0ms
Speed: 2.0ms preprocess, 17.0ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 class_2, 16.5ms
Speed: 2.0ms preprocess, 16.5ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 class_7, 16.4ms
Speed: 2.7ms preprocess, 16.4ms inference, 1.1ms postprocess per image at

In [9]:
import yaml

# Path ke direktori dataset output
dataset_dir = '/kaggle/working/dataset2'

# Definisikan konfigurasi YAML
config = {
    'path': dataset_dir,
    'train': f'{dataset_dir}/train',
    'val': f'{dataset_dir}/test',
    'nc': 43,  # Jumlah kelas yang Anda miliki
    'names': ['class0', 'class1', 'class2', 'class3', 'class4', 'class5', 
              'class6', 'class7', 'class8', 'class9', 'class10', 'class11', 
              'class12', 'class13', 'class14', 'class15', 'class16', 'class17', 
              'class18', 'class19', 'class20', 'class21', 'class22', 'class23', 
              'class24', 'class25', 'class26', 'class27', 'class28', 'class29', 
              'class30', 'class31', 'class32', 'class33', 'class34', 'class35', 
              'class36', 'class37', 'class38', 'class39', 'class40', 'class41', 
              'class42']  # Nama kelas sesuai dengan urutan indeks
}

# Tulis konfigurasi ke dalam file YAML
yaml_file = '/kaggle/working/dataset2/dataset.yaml'
with open(yaml_file, 'w') as file:
    yaml.dump(config, file)

print("YAML file created successfully.")


YAML file created successfully.


In [10]:
import os
from ultralytics import YOLO

data_path = '/kaggle/working/dataset2/dataset.yaml'
model_path = '/kaggle/input/trafficsign/other/yolo/1/best11.pt'

model = YOLO(model_path)

model.train(
    data=data_path,
    epochs=5000,
    imgsz=640,
    batch=64,
    fliplr=0.0,    # Hindari flipping horizontal (mirroring)
    flipud=0.0,    # Hindari flipping vertikal (mirroring)
    degrees=0.0,   # Hindari rotasi
)


Ultralytics YOLOv8.2.32 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/kaggle/input/trafficsign/other/yolo/1/best11.pt, data=/kaggle/working/dataset2/dataset.yaml, epochs=5000, time=None, patience=100, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=F

100%|██████████| 755k/755k [00:00<00:00, 15.8MB/s]
2024-06-16 18:37:19,113	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-06-16 18:37:19,862	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.



                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256

100%|██████████| 6.23M/6.23M [00:00<00:00, 71.8MB/s]


AMP: checks passed ✅


...

                   all        111        179      0.473      0.477      0.466      0.353
EarlyStopping: Training stopped early as no improvement observed in last 100 epochs. Best results observed at epoch 249, best model saved as best.pt.
To update EarlyStopping(patience=100) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



349 epochs completed in 1.132 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 22.6MB
Optimizer stripped from runs/detect/train/weights/best.pt, 22.6MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.32 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11142225 parameters, 0 gradients, 28.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.32it/s]


                   all        111        179      0.472      0.519      0.519      0.408
                class0          2          2          1          0      0.375      0.338
                class1         15         15      0.603      0.912       0.78      0.676
                class2         10         10      0.342        0.9      0.591      0.492
                class3          4          6      0.405      0.833      0.608      0.483
                class4          3          3      0.215      0.667       0.26      0.243
                class5          3          5     0.0583        0.2      0.135     0.0895
                class6          2          4      0.476      0.918      0.745      0.457
                class7          3          5      0.208      0.578      0.462      0.391
                class8          5          8      0.864        0.8      0.831      0.689
                class9          1          1      0.251          1      0.995      0.895
               class1

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 25, 26, 28, 29, 30, 31, 32, 34, 35, 38, 39, 40, 41, 42])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f0fce9511b0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.